# 检索模型
## 检索模型概述
一篇文档如果被判定和一个查询是同一个主题，那么它们是主题相关的。

用户相关性考虑用户在判定相关性时涉及的所有因素。这些因素包括文档的年代、语言、目标受众、新颖性等。

相关性是二元还是多元的：二元相关性就是简单地判定一篇文档是相关的还是非相关的。

### 布尔检索模型
假设在检索到的集合中，所有文档关于相关性都是等价的，同时也假设了相关性是二元的。

### 向量空间模型
一篇文档Di表示为索引词项的一个向量：$D_i=((d_{i1},d_{i2}, ..., d_{it}))$

可以表示为一个词项权值的矩阵：

$
\begin{matrix}
       & term1 & term2 & \cdots & termt \\
doc1 & d_{11} & d_{12} & \cdots & d_{1t} \\
doc2 & d_{21} & d_{22} & \cdots & d_{2t} \\
\vdots & \vdots & \vdots & \cdots & \vdots \\
docn & d_{n1} & d_{n2} & \cdots & d_{nt}
\end{matrix}
$

文档可以通过计算表示文档和查询的点之间的距离来进行排序。

余弦相关性：$cos(D_i,Q)=\frac{\sum_{j=1}^t d_{ij} \cdot q_j}{\sqrt{\sum_{j=1}^t d_{ij}^2 \cdot \sum_{j=1}^t q_j^2}}$

tf表示文档中词项的频率，反映了一个词项在文档Di（或查询项）中的重要性。$tf_{ik}=\frac{f_{ik}}{\sum_{j=1}^t f_{ij}}$

idf反映了文档数据集中词项重要性。如果在其中出现过一个词项的文档越多，这个词项在文档之间就越没有区分性，也就对检索越没有用。$idf_k=log\frac{N}{n_k}$

Rocchio(1971)：一种根据用户判定的相关文档来修改查询项的算法，这个算法会使得相关文档向量的平均向量和非相关文档向量的平均向量之间的差异化最大。$q_j'=\alpha \cdot q_j=\beta \cdot \frac{1}{|Rel|}\sum_{D_i in Rel}d_{ij}- \gamma \cdot \frac{1}{|Norel|}\sum_{D_i \in Norel}d_{ij}$
  - $q_j$查询项j的初始权重
  - Rel用户选定的相关文档集合
  - Norel非相关文档集合
  - |.|返回一个集合的大小
  - $d_{ij}$ 是文档i中第j个词项的权值
  - $\alpha,\beta,\gamma$控制每个部分影响的参数，合理的数值分别是8，16，4

## 概率模型
### 将信息检索作为分类问题
当P(R|D)>P(NR|D)时，判定文档D是相关的，其中P(R|D)是相关性的条件概率，P(NR|D)是非相关性的条件概率。

$P(R|D)=\frac{P(D|R)P(R)}{P(D)}$

P（R）是相关性的先验概率（任何文档都是相关的可能性）。将判定一篇文档相关的条件（P(R|D)>P(NR|D)）代入：

$\frac{P(D|R)}{P(D|NR)}>\frac{P(NR)}{P(R)}$

左边部分称为似然比。搜索引擎只需要排序文档，不需要做出分类判断。如果采用似然比作为分值，排序较高的是那些对于属于相关集合具有较高似然值的文档。

假设文档由词项组成，相关文档和不相关文档表示为词项的集合：
- 文档表示为一组二元向量特征，$D=(d_1,d_2,...,d_t)$，其中$d_i$表示词项i出现在文档中，反之为0。
- 朴素贝叶斯假设：词项独立性，这意味着可以 $\prod_{i=1}^t P(d_i|R)$ 来估计P(D|R)，类似可以计算P(D|NR)。

$\frac{P(D|R)}{P(D|NR)}=\prod_{i:d_i=1}\frac{p_i}{s_i} \cdot \prod_{i:d_i=0}\frac{1-p_i}{1-s_i}$
- $p_i$是词项i在相关集合的某篇文档中出现（出现为1）的概率。
- $s_i$是词项i在不相关集合的某篇文档中出现（出现为1）的概率。
- $\prod_{i:d_i=1}$ 文档中值是1的词项概率的连乘。

数学推导：

$\prod_{i:d_i=1}\frac{p_i}{s_i} \cdot (\prod_{i:d=1}\frac{1-s_i}{1-p_i} \cdot \prod_{i:d_i=1}\frac{1-p_i}{1-s_i})\prod_{i:d=0}\frac{1-p_i}{1-s_i}$

$=\prod_{i:d_i=1}\frac{p_i(1-s_i)}{s_i(1-p_i)} \cdot \prod_i\frac{1-p_i}{1-s_i}$

第二项连乘涵盖所有词项，也能覆盖所有文档，所以对于排序能够忽略掉。取对数控制精度问题:

$\sum_{i:d_i=1}log\frac{p_i(1-s_i)}{s_i(1-p_i)}$

假设，在其他信息存在的基础上，查询项中那个没有出现的词项对于相关文档和非相关文档具有相同的出现概率（即pi=si）。也就是说，给定一个文档，文档的分值可以简单地将匹配到的词项概率求和即可。

如果没有相关集合的其他信息，可以额外假设pi是一个常数，si可以被近似估计为整个文档数据集中的词项出现情况（依据是相关文档的数量远小于整体文档集合的大小），假定pi=0.5，给定词项i的权值为

$log\frac{0.5(1-\frac{n_i}{N})}{\frac{n_i}{N}(1-0.5)}=log\frac{N-n_i}{n_i}$

假定：
- N整个文档数据集中所有文档数量
- R和这个查询相关的文档数量
- ni包含词项i的文档数量
- ri包含词项i的相关文档数量

则:
- $p_i=\frac{r_i}{R}$
- $s_i=\frac{n_i-r_i}{N-R}$

在每个数值上加0.5，整体数值上加1:
- $p_i=\frac{r_i+0.5}{R+1}$
- $s_i=\frac{n_i-r_i+0.5}{N-R+1}$

$\sum_{i:d_i=q_i=1}log\frac{(r_i+0.5)/(R-r_i+0.5)}{(n_i-r_i+0.5)/(N-n_i-R+r_i+0.5)}$

### BM25排序算法
通过加入文档权值和查询项权值，拓展了二元独立模型的得分函数：
$\sum_{i \in Q}log\frac{(r_i+0.5)/(R-r_i+0.5)}{(n_i-r_i+0.5)/(N-n_i-R+r_i+0.5)} \cdot \frac{(k_1+1)f_i}{K+f_i} \cdot \frac{(k_2+1)qf_i}{k_2+qf_i}$

- $f_i$词项i在文档中的频率
- ${qf}_i$词项i在查询中的频率
- k1,k2,K经验设定参数

## 基于排序的语言模型
一元语言模型就是语言中词汇的概率分布。例如，如果文档数据集中只包含5个不同的词语，这个集合一个可能的语言模型是(0.2,0.1,0.35.0.25,0.1)，其中每个数值表示词语出现的概率。如果将每个文档看成是词汇的一个序列，那么语言模型的概率就是预测序列中下一个词语的概率。

n元语言模型使用更长序列来预测词语。一个n元模型预测词语时考察前面的n-1个词语。

将文档表示为语言模型后，同样能够将查询的主题表示为语言模型。这种情况下，语言模型是主题的一种表示。这种主题是搜索信息的人在写下查询时头脑中所想的内容。这就导致了三种基于语言模型的检索概率值：一种是基于从文档语言模型生成查询文本的概率；一种是基于从查询项语言模型生成文档文本的概率；一种是基于对比查询语言模型和文档主题语言模型的结果。

### 查询项似然排序
$P(D|Q) \triangleq P(Q|D)P(D)$

- $\triangleq$ 排序等价
- P(D)文档先验概率
- P(Q|D)给定文档后查询的似然函数

P(D)被假设是始终如一（对所有文档都一样），不影响排序结果。所以检索模型通过P(Q|D)来排序文档，这个概率值采用文档一元语言模型来计算：

$P(Q|D)=\prod_{i=1}^nP(q_i|D)$
- qi是查询项中的词，查询项中有n和词。

估计语言模型概率值：

$P(q_i|D)=\frac{f_{q_i,D}}{|D|}$
- $f_{q_i,D}$ 是词语qi在文档集合D中出现次数。
- |D|表示D中词语的数量。

平滑技术用于避免这种估计问题以及数据稀疏问题，一般的方法是降低（或者打折）文档文本中出现词语的估计概率，并对文本中未出现的词语赋给估计的“剩余”概率。未出现词语的概率通常都是基于整个文档数据集中词语的出现频率来进行估计。如果P(qi|C)是文档集合C的数据集语言模型中词语i的出现概率，那么文档中未出现词语的估计概率为 $\alpha_DP(q_i|C)$ ，其中 $\alpha_D$ 是控制赋予未见词语概率的系数。为了保证概率值和为1，文档中一个出现过的词语的概率估计为:

 $(1-\alpha_D)P(q_i|D)+\alpha_DP(q_i|C)$

Jelinek-Mercer平滑方法估计 $P(q_i|D)$

$P(q_i|D)=(1-\lambda)\frac{f_{q_i,D}}{|D|}+\lambda\frac{c_{q_i}}{|C|}$
- $\alpha_D=\lambda$
- 估计词语qi的概率为 $\frac{C_{qi}}{|C|}$
- $C_{qi}$ 是文档数据集中查询词出现的次数
- |C|是集合中所有词语出现的次数总和

文档得分公式： $logP(Q|D)=\sum_{i=1}^nlog(1-\lambda)\frac{f_{q_i,D}}{|D|}+\lambda\frac{c_{qi}}{|C|}$

狄利克雷平滑方法估计 $P(q_i|D)$

$p(q_i|D)=\frac{f_{q_i,D}+\mu\frac{c_{qi}}{|C|}}{|D|+\mu}$
- $\alpha_D=\frac{\mu}{|D|+\mu}$

文档得分公式： $logP(Q|D)=\sum_{i=1}^nlog\frac{f_{q_i,D}+\mu\frac{c_{qi}}{|C|}}{|D|+\mu}$

### 相关性模型和伪相关反馈
如果能够从一个查询估计一个相关性模型，那么就能将这个语言模型和文档模型直接进行对比。文档会根据文档模型和相关性模型的相似程度进行排序。

Kullback-Leibler分散度（KL-分散度），能够度量两个概率分布的差异。给定真实的概率分布P以及另外一个用于估计P的概率分布Q，KL-分散度定义为：

$KL(P||Q)=\sum_xP(x)log\frac{P(x)}{Q(x)}$

利用相关性模型进行排序的概括步骤：
1. 根据对查询Q的查询似然得分对文档排序；
2. 选择排序靠前的某个数目的文档构成集合C；
3. 利用估计概率P(w,q1,...,qn)来计算相关性模型概率P(w|R)；
4. 利用KL-分散度来对文档进行再次排序：

$\sum_wP(w|R)logP(w|D)$

## 复杂查询与证据整合
高效检索需要将与文档可能相关的许多片段的证据组合起来。

贝叶斯网络是一种用于区分一组事件和事件之间依赖关系的概率模型。这种网络是有向无环图(DAG)。图中的节点表示带有一组可能输出的事件，弧表示事件之间的依存关系。

## 机器学习和信息检索
### 排序学习
Ranking SVM的输入是针对一组查询的偏序排序信息的一组训练集合

$(q_1,r_1),(q_2,r_2), \cdots,(q_n,r_n)$

其中qi是一个查询，ri是所需排序的文档关于查询的部分排序信息或相关性级别。这意味着，如果去、文档$d_a$应该比$d_b$排序更高，那么$(d_a,d_b) \in r_i$，否则 $(d_a,d_b) \notin r_i$。

学习一个线性排序函数$\vec{w} \cdot \vec{d_a}$，其中$\vec{w}$是一个用于通过学习调整的权值向量，$\vec{d_a}$是文档$d_a$的特征表示向量。

$mini:\frac{1}{2} \vec{w} \cdot \vec{w}+C\sum\xi_{i,j,k}$

subject to:

$\forall (d_i,d_j) \in r_1:\vec{w} \cdot \vec{d_i} > \vec{w} \cdot \vec{d_j} + 1 - \xi_{i,j,1}$

$\forall (d_i,d_j) \in r_n:\vec{w} \cdot \vec{d_i} > \vec{w} \cdot \vec{d_j} + 1 - \xi_{i,j,n}$

$\forall i \forall j \forall k:\xi_{i,j,k} \ge 0$

### 主题模型和词汇不匹配
生成一篇文档的LDA过程：

1. 对每个文档D，根据参数$\alpha$在一个狄利特雷分布中选择一个多项式分布$\theta_D$。
1. 对文档D中的每个词语位置：
    1. 从多项式分布$\theta_D$中选择一个主题z。
    1. 从$P(w|z,\beta)$中选择一个词语w，$P(w|z,\beta)$是给定主题z和参数$\beta$时的多项式条件概率。

搜索应用采用LDA的主要问题：估计模型中的概率比较费事。
